In [ ]:
## GDRIVE CONNETIONS
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data_dir = "/content/drive/MyDrive/Workspace-COLAB/biom-task3/classifier-keras/dataset/awe-train-test-val/train"
valid_data_dir = "/content/drive/MyDrive/Workspace-COLAB/biom-task3/classifier-keras/dataset/awe-train-test-val/val"

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

https://github.com/yuyang-huang/keras-inception-resnet-v2

https://github.com/sabarish244/Face-Recognition-using-Transfer-learning/blob/master/transfer_learning_collab.ipynb

https://keras.io/api/applications/

https://keras.io/api/preprocessing/image/

In [ ]:
# DATASET

In [ ]:
IMAGE_SIZE = 224
num_classes = 100

img_rows = IMAGE_SIZE
img_cols = IMAGE_SIZE

In [ ]:
# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
valid_datagen = ImageDataGenerator(rescale=1./255)
 

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols))
 
valid_generator = valid_datagen.flow_from_directory(
        valid_data_dir,
        target_size=(img_rows, img_cols))

Found 800 images belonging to 100 classes.
Found 100 images belonging to 100 classes.


In [ ]:
# MODEL SETUP

In [ ]:
# model accepts 224 x 224

# Re-loads the model without the top or FC layers
base_model = InceptionResNetV2(weights='imagenet', include_top = False , input_shape=(img_rows,img_cols,3) , pooling='avg')



219062272/219055592 [==============================] - 1s 0us/step


In [ ]:
/

()

In [ ]:
# ADD MODEL TOP LAYERS

In [ ]:
from keras.models import Model
from keras.layers import Dense
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model


In [ ]:


outputs = Dense(num_classes, activation='softmax')(base_model.output)
newmodel = Model(base_model.inputs, outputs)

print(newmodel.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
# TRAINING

In [ ]:
path_model_checkpoint= "/content/drive/MyDrive/Workspace-COLAB/biom-task3/tf-keras-inceptionResnetV2/ear-recognizer-dense.h5"
path_model_save = "/content/drive/MyDrive/Workspace-COLAB/biom-task3/tf-keras-inceptionResnetV2/ear-recognizer-dense.h5"

In [ ]:
steps_per_epoch=50
num_epochs=100

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
                     
checkpoint = ModelCheckpoint(path_model_checkpoint,
                             monitor="val_accuracy",
                             mode="min",
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_accuracy',
                          verbose=1, 
                          patience=5,
                          restore_best_weights=True)

# we put our call backs into a callback list
#callbacks = [earlystop, checkpoint]
callbacks = [checkpoint]

# We use a very small learning rate 
newmodel.compile(loss = 'binary_crossentropy',
              optimizer = Adam(lr = 0.001),
              metrics = ['accuracy'])

history = newmodel.fit_generator(
    generator=train_generator,
    epochs = num_epochs,
    callbacks = callbacks,
    validation_data = valid_generator)
newmodel.save(path_model_save)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
25/25 [==============================] - 174s 6s/step - loss: 0.2596 - accuracy: 0.0080 - val_loss: 0.1045 - val_accuracy: 0.0100

Epoch 00001: val_accuracy improved from inf to 0.01000, saving model to /content/drive/MyDrive/Workspace-COLAB/biom-task3/tf-keras-inceptionResnetV2/ear-recognizer-dense.h5
Epoch 2/100
25/25 [==============================] - 16s 638ms/step - loss: 0.0563 - accuracy: 0.0499 - val_loss: 0.0987 - val_accuracy: 0.0100

Epoch 00002: val_accuracy did not improve from 0.01000
Epoch 3/100
25/25 [==============================] - 16s 639ms/step - loss: 0.0520 - accuracy: 0.0838 - val_loss: 0.1102 - val_accuracy: 0.0000e+00

Epoch 00003: val_accuracy improved from 0.01000 to 0.00000, saving model to /content/drive/MyDrive/Workspace-COLAB/biom-task3/tf-keras-inceptionResnetV2/ear-recognizer-dense.h5
Epoch 4/100
25/25 [==============================] - 17s 657ms/step - loss: 0.0456 - accuracy: 0.1905 - val_loss: 0.1116 - val_accuracy: 0.0200

Epoch 00004: 

In [ ]:
from keras.models import load_model
from tensorflow.keras.applications.inception_resnet_v2 import decode_predictions
from keras.preprocessing import image
his_img = image.load_img("/content/drive/MyDrive/Workspace-COLAB/biom-task3/classifier-keras/dataset/awe-train-test-val/test/004/04.png", target_size=(224,224))
import numpy as np
his_img = image.img_to_array(his_img)
exp_img = np.expand_dims(his_img, axis=0)
model = load_model("/content/drive/MyDrive/Workspace-COLAB/biom-task3/tf-keras-inceptionResnetV2/ear-recognizer-dense.h5")
pred = model.predict(exp_img)
pred

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)

In [ ]:
np.argmax(pred)

25